In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset

import os
import sys
import tables
import pickle
import copy

import pandas as pd
import numpy as np
from numba import cuda

from sklearn.model_selection import KFold
from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from scipy.stats import pearsonr

In [2]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


# load the data

In [3]:
def identify_constant_columns(data):

    constant_cols_list = []

    for index in range(data.shape[1]):

        col = data[:, index]
        is_unique = len(np.unique(col)) == 1

        if is_unique:
            constant_cols_list.append(index)
            
    return constant_cols_list

In [4]:
def scale_data_and_pca(tr_x, val_x, dataset, pca=True):
    
    scaler = StandardScaler()
    scaler.fit(tr_x)
    tr_x = scaler.transform(tr_x).astype(np.float32)
    val_x = scaler.transform(val_x).astype(np.float32)
    
    if pca:
        file_name = f"pca_{dataset}.p"

        if os.path.exists(file_name):
            pca = pickle.load(open(file_name, "rb"))
        else:
            pca = PCA(tr_x.shape[1])
            pca.fit(tr_x)
            pickle.dump(pca, open(file_name, "wb"))

        tr_x = pca.transform(tr_x)
        val_x = pca.transform(val_x)
    
    return tr_x, val_x

In [5]:
def merge_data_with_metadata(train_x, test_x, metadata, index_train, index_test, index_meta_data):
    
    metadata = pd.DataFrame(np.hstack((np.expand_dims(index_meta_data, 1), metadata)))
    
    train_x = pd.DataFrame(np.hstack((np.expand_dims(index_train, 1), train_x))).merge(metadata, how="inner", left_on=0, right_on=0)
    test_x = pd.DataFrame(np.hstack((np.expand_dims(index_test, 1), test_x))).merge(metadata, how="inner", left_on=0, right_on=0)
    
    return train_x, test_x

In [6]:
def load_metadata():
    
    metadata = pd.read_csv("/kaggle/input/open-problems-multimodal/metadata.csv")
    index = metadata["cell_id"]
    
    enc_metadata = OneHotEncoder(handle_unknown='ignore')
    metadata = enc_metadata.fit_transform(metadata[["day", "donor", "cell_type"]].values).toarray()
    
    return index, metadata

In [7]:
# get the paths to load the data

def load_dataset(data_name):

    dir_path = "/kaggle/input/open-problems-multimodal/"

    if data_name == "citeseq":
        train_x_path = os.path.join(dir_path,"train_cite_inputs.h5")
        train_y_path = os.path.join(dir_path,"train_cite_targets.h5")
        test_x_path = os.path.join(dir_path,"test_cite_inputs.h5")

    elif data_name == "multiome":
        train_x_path = os.path.join(dir_path,"train_multi_inputs.h5")
        train_y_path = os.path.join(dir_path,"train_multi_targets.h5")
        test_x_path = os.path.join(dir_path,"test_multi_inputs.h5")

    else:
        raise NameError(f"{data_name} is not a valid name: choose between 'siteseq' and 'multiome'")
        
    train_x = pd.read_hdf(train_x_path)
    train_y = pd.read_hdf(train_y_path)
    test_x = pd.read_hdf(test_x_path)
    
    index_train = train_x.index
    index_test = test_x.index
    
    train_x, train_y, test_x = train_x.to_numpy(), train_y.to_numpy(), test_x.to_numpy()
    
    # some columns in the training sets are constant (zeros), remove them
    constant_columns_train = identify_constant_columns(train_x)
    train_x = train_x[:, [i for i in range(train_x.shape[1]) if i not in constant_columns_train]]
    test_x = test_x[:, [i for i in range(test_x.shape[1]) if i not in constant_columns_train]]
    
    return train_x, train_y, test_x, index_train, index_test

In [8]:
dataset = "citeseq"

add_metadata = False

In [9]:
train_x, train_y, test_x, index_train, index_test = load_dataset(dataset)

In [39]:
# if add_metadata:

#     index_meta_data, meta_data = load_metadata()
#     train_x, test_x = merge_data_with_metadata(train_x, test_x, meta_data, index_train, index_test, index_meta_data)
    
#     train_x = np.array(train_x)[:, 1:]
#     test_x = np.array(test_x)[:, 1:]

# Pytorch utils

In [14]:
# training loop for each epoch

def train(dataloader, model, loss_fn, optimizer, device):
    
    num_batches = len(dataloader)
    model.train()
    loss = 0
    corr = 0
    
    for data_dic in dataloader:
        X, y = data_dic['x'].to(device), data_dic['y'].to(device)

        # Compute prediction error
        pred = model(X)
        train_loss = loss_fn(pred, y)
        
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()
            
        loss += train_loss.item()
        corr += np.mean([pearsonr(pred[i].cpu().detach().numpy(), y[i].cpu().detach().numpy())[0] for i in range(pred.shape[0])])
    
    loss /= num_batches
    corr /= num_batches
    
    return loss, corr


# validation loop for each epoch
def val(dataloader, model, loss_fn, device):
    
    pred_val_list = []
    num_batches = len(dataloader)
    model.eval()
    loss = 0
    corr = 0
    
    with torch.no_grad():
        for data_dic in dataloader:
            X, y = data_dic['x'].to(device), data_dic['y'].to(device)
            
            pred = model(X)
            pred_val_list.append(pred.cpu().detach().numpy())
            val_loss = loss_fn(pred, y)
                
            loss += val_loss.item()    
            corr += np.mean([pearsonr(pred[i].cpu().detach().numpy(), y[i].cpu().detach().numpy())[0] for i in range(pred.shape[0])])
            
    loss /= num_batches
    corr /= num_batches
    
    return loss, corr, pred_val_list

# make predictions on the test set (dataloader is only made of x)

def test(dataloader, model, loss_fn, device):
    
    pred_list = []
    num_batches = len(dataloader)
    model.eval()
    
    with torch.no_grad():
        for data_dic in dataloader:
            
            X = data_dic['x'].to(device)
            pred = model(X)
            pred_list.append(pred)
    
    return pred_list

In [15]:
# Dataset class that loads the data and prepare it for the pytorch dataloader

class CompetitionDataset(Dataset):

    def __init__(self, data_tuple, mode='train'):
        self.mode = mode
        
        if self.mode == "train":
            assert len(data_tuple) == 2, "`data_tuple` should have lenght 2"
            data_x, data_y = data_tuple
        elif self.mode == "test":
            assert len(data_tuple) == 1, "`data_tuple` should have length 1"
            data_x = data_tuple[0]
        else:
            raise NameError(f"{self.mode} is not a valid mode: choose between 'train' and 'test'")

        self.filenames = dict()
        self.filenames['x'] = data_x
        
        if self.mode == "train":
            self.filenames['y'] = data_y

    def __getitem__(self, index):
        batch = dict()
        
        batch['x'] = torch.from_numpy(self.filenames['x'][index])
        
        if self.mode == "train":
            batch['y'] = torch.from_numpy(self.filenames['y'][index])
        
        return batch

    def __len__(self):
        return len(self.filenames['x'])

In [16]:
def pytorch_dataset_and_dataloader(tr_x, tr_y, val_x, val_y,  batch_size = 256, mode = "train"):

    train_dataset = CompetitionDataset((tr_x, tr_y), mode)
    val_dataset = CompetitionDataset((val_x, val_y), mode) # here "train" is also used for validation
    
    train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size, drop_last=False)
    val_dataloader = DataLoader(val_dataset, shuffle=True, batch_size=batch_size, drop_last=False)
    
    return train_dataloader, val_dataloader

# Pytorch simple MLP

In [17]:
# Define model
class NeuralNetwork(nn.Module):
    def __init__(self, n_shared_hidden_layers, shared_hidden_size_list, n_non_shared_hidden_layers, non_shared_hidden_size_list, 
                 dropout, output_size, use_multi_head=False):
        super(NeuralNetwork, self).__init__()
        
        assert len(shared_hidden_size_list) == n_shared_hidden_layers, f"`hidden_size_list` should have length {n_shared_hidden_layers}"
        assert len(non_shared_hidden_size_list) == n_non_shared_hidden_layers, f"`hidden_size_list` should have length {n_non_shared_hidden_layers}"
        
        self.n_shared_hidden_layers = n_shared_hidden_layers
        self.shared_hidden_size = shared_hidden_size_list
        self.n_non_shared_hidden_layers = n_non_shared_hidden_layers
        self.non_shared_hidden_size = non_shared_hidden_size_list
        self.dropout = dropout
        self.output_size = output_size
        self.use_multi_head = use_multi_head
        
        # hidden layers
        self.shared_hidden_layers_list = nn.ModuleList()
        for l in range(self.n_shared_hidden_layers):
            if l == 0:
                self.shared_hidden_layers_list.append(nn.LazyLinear(self.shared_hidden_size[l]))
            else:
                self.shared_hidden_layers_list.append(nn.Linear(self.shared_hidden_size[l-1], self.shared_hidden_size[l]))
            self.shared_hidden_layers_list.append(nn.ReLU())
            self.shared_hidden_layers_list.append(nn.Dropout(self.dropout))
            self.shared_hidden_layers_list.append(nn.BatchNorm1d(self.shared_hidden_size[l]))
            
        self.shared_hidden_layers_list = nn.Sequential(*self.shared_hidden_layers_list)
        
        if self.use_multi_head:
            self.non_shared_hidden_layers = nn.ModuleList(nn.ModuleList() for _ in range(self.output_size))
            for i in range(self.output_size):
                for l in range(self.n_non_shared_hidden_layers):
                    if l == 0:
                        self.non_shared_hidden_layers[i].append(nn.Linear(self.shared_hidden_size[-1], self.non_shared_hidden_size[l]))
                    else:
                        self.non_shared_hidden_layers[i].append(nn.Linear(self.non_shared_hidden_size[l-1], self.non_shared_hidden_size[l]))
                    self.non_shared_hidden_layers[i].append(nn.ReLU())
                    self.non_shared_hidden_layers[i].append(nn.Dropout(self.dropout))
                    self.non_shared_hidden_layers[i].append(nn.BatchNorm1d(self.non_shared_hidden_size[l]))
                self.non_shared_hidden_layers[i].append(nn.Linear(self.non_shared_hidden_size[-1], 1))
                
                self.non_shared_hidden_layers[i] = nn.Sequential(*self.non_shared_hidden_layers[i])
        
        self.output_layer = nn.Linear(self.shared_hidden_size[-1], output_size)
        
    def forward(self, x):
        
        shared_x = self.shared_hidden_layers_list(x)
            
        if self.use_multi_head:
            output = torch.empty(size=(shared_x.shape[0], self.output_size)).to("cuda")
            for i in range(self.output_size):
                not_shared_x = shared_x
                not_shared_x = self.non_shared_hidden_layers[i](not_shared_x)
                output[:, i] = torch.squeeze(not_shared_x)
        else:
            output = self.output_layer(shared_x)
        
        return output

In [18]:
pca = True
n_components_to_keep = 100

In [19]:
# network parameters

n_shared_hidden_layers = 2
n_non_shared_hidden_layers = 2

dropout = 0.5

shared_hidden_size_list = [1024, 1024] 
non_shared_hidden_size_list = [1024, 512] 

output_size = train_y.shape[1]

In [20]:
use_multi_head = True

In [21]:
lr = 1e-3
batch_size = 512

In [22]:
n_cv_fold = 5
n_epoch = 20

In [34]:
# cross validation

fold_number = 0

for train_index_outer, val_index_outer in KFold(n_cv_fold, shuffle=True, random_state=0).split(train_x):
    print(f"computing for fold number {fold_number+1}")
    
    tr_x, tr_y = train_x[train_index_outer], train_y[train_index_outer]
    val_x, val_y = train_x[val_index_outer], train_y[val_index_outer]
    
    tr_x, val_x =  scale_data_and_pca(tr_x, val_x, dataset, pca)
    tr_x = tr_x[:, :n_components_to_keep]
    val_x = val_x[:, :n_components_to_keep]
    
    train_dataloader, val_dataloader = pytorch_dataset_and_dataloader(tr_x, tr_y, val_x, val_y, batch_size, "train")
    
    # instantiate a newly initialized model and a new optimize (adam aggregates gradients so it needs to be reset)
    model = NeuralNetwork(n_shared_hidden_layers, shared_hidden_size_list, n_non_shared_hidden_layers, 
                          non_shared_hidden_size_list, dropout, output_size, use_multi_head).to(device)
    loss_fn = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    
    for epoch in range(n_epoch):
        
        # train for one full epoch and gather training_loss
        loss_train, corr_train = train(train_dataloader, model, loss_fn, optimizer, device)
        # compute the validation loss after the epoch
        loss_val, corr_val, pred_val_list = val(val_dataloader, model, loss_fn, device)
        
        print(f"OUTER LOOP epoch {epoch}")
        print(f"training loss: {loss_train}, corr train: {corr_train}, validation_loss: {loss_val}, corr val: {corr_val}")
        
    break

    fold_number += 1

computing for fold number 1


/srv/conda/envs/saturn/lib/python3.9/site-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


OUTER LOOP epoch 0
training loss: 9.193361662529611, corr train: 0.5281938819416625, validation_loss: 3.1056170974458968, corr val: 0.8756526305186643
OUTER LOOP epoch 1
training loss: 3.154270174267056, corr train: 0.866099737489893, validation_loss: 2.4696242128099715, corr val: 0.897057650242531
OUTER LOOP epoch 2
training loss: 2.8757982382903227, corr train: 0.878629707941741, validation_loss: 2.424633707318987, corr val: 0.8987493494876678
OUTER LOOP epoch 3
training loss: 2.7566060633272738, corr train: 0.8836650708735669, validation_loss: 2.3993845752307346, corr val: 0.899363202799381
OUTER LOOP epoch 4
training loss: 2.6669177386137815, corr train: 0.8870770573189948, validation_loss: 2.398426490170615, corr val: 0.8996271371875565
OUTER LOOP epoch 5
training loss: 2.603389888196378, corr train: 0.8894401262301211, validation_loss: 2.365670212677547, corr val: 0.9003941112505461
OUTER LOOP epoch 6
training loss: 2.558800740284963, corr train: 0.8911160914549549, validation_lo

In [27]:
torch.cuda.empty_cache()

In [ ]:
# cross validation

fold_number = 0

for train_index_outer, val_index_outer in KFold(n_cv_fold, shuffle=True, random_state=0).split(train_x):
    print(f"computing for fold number {fold_number+1}")
    
    tr_x, tr_y = train_x[train_index_outer], train_y[train_index_outer]
    val_x, val_y = train_x[val_index_outer], train_y[val_index_outer]
    
    tr_x, val_x =  scale_data_and_pca(tr_x, val_x, dataset, pca)
    tr_x = tr_x[:, :n_components_to_keep]
    val_x = val_x[:, :n_components_to_keep]
    
    train_dataloader, val_dataloader = pytorch_dataset_and_dataloader(tr_x, tr_y, val_x, val_y, batch_size, "train")
    
    # instantiate a newly initialized model and a new optimize (adam aggregates gradients so it needs to be reset)
    model = NeuralNetwork(n_shared_hidden_layers, shared_hidden_size_list, n_non_shared_hidden_layers, 
                          non_shared_hidden_size_list, dropout, output_size, use_multi_head).to(device)
    loss_fn = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    
    for epoch in range(1):
        
        # train for one full epoch and gather training_loss
        loss_train, corr_train = train(train_dataloader, model, loss_fn, optimizer, device)
        # compute the validation loss after the epoch
        loss_val, corr_val, pred_val_list = val(val_dataloader, model, loss_fn, device)
        
        print(f"OUTER LOOP epoch {epoch}")
        print(f"training loss: {loss_train}, corr train: {corr_train}, validation_loss: {loss_val}, corr val: {corr_val}")
    
    pred_train = model(torch.from_numpy(val_x).to(device))
    pred_train = pred_train.cpu().detach().numpy()
    pred_train = np.hstack((val_x, pred_train))
        
    for train_index_inner, val_index_inner in KFold(10, shuffle=True, random_state=0).split(pred_train):
        
        tr_x_inner, tr_y_inner = pred_train[train_index_inner], val_y[train_index_inner]
        val_x_inner, val_y_inner = pred_train[val_index_inner], val_y[val_index_inner]
        
        tr_x, val_x =  scale_data_and_pca(tr_x_inner, val_x_inner, dataset, False)
        
        train_dataloader, val_dataloader = pytorch_dataset_and_dataloader(tr_x_inner, tr_y_inner, val_x_inner, val_y_inner, batch_size, "train")
        
        model = NeuralNetwork(n_shared_hidden_layers, shared_hidden_size_list, n_non_shared_hidden_layers, 
                          non_shared_hidden_size_list, dropout, output_size, use_multi_head).to(device)
        loss_fn = nn.MSELoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=lr)
        
        for epoch in range(50):
        
            # train for one full epoch and gather training_loss
            loss_train, corr_train = train(train_dataloader, model, loss_fn, optimizer, device)
            # compute the validation loss after the epoch
            loss_val = val(val_dataloader, model, loss_fn, device)
            
            print(f"INNER LOOP epoch {epoch}")
            print(f"training loss: {loss_train}, corr train: {corr_train}, validation_loss: {loss_val}")

    fold_number += 1
        
    break

    fold_number += 1

computing for fold number 1
